In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential , Model
from keras.layers import MaxPooling2D , Conv2D , Dense , Activation , Dropout , Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.regularizers import l2, l1

In [7]:
data = pd.read_json('G:/Software/Machine learning/Datasets/Planet json/planesnet.json')
data

,data,labels,locations,scene_ids
0,"[206, 195, 187, 183, 177, 175, 174, 193, 198, ...",1,"[-118.40497658522878, 33.940618514147936]",20170620_175442_0e30
1,"[215, 209, 200, 196, 192, 197, 205, 168, 155, ...",1,"[-122.392469714, 37.6176425378]",20161212_180859_0e30
2,"[204, 214, 220, 219, 213, 205, 198, 193, 199, ...",1,"[-122.397578597, 37.6209247852]",20170524_181349_0e2f
3,"[179, 174, 179, 178, 173, 170, 168, 168, 168, ...",1,"[-122.214849831, 37.7203378331]",20161110_180707_0e1f
4,"[222, 222, 218, 214, 208, 205, 207, 206, 206, ...",1,"[-117.862173435, 33.6796854072]",20160813_184932_0c64
...,...,...,...,...
31995,"[217, 197, 206, 221, 219, 200, 202, 222, 231, ...",0,"[-117.51619965614212, 34.08254781774721]",20170726_174731_0f34
31996,"[184, 198, 218, 222, 220, 216, 216, 215, 210, ...",0,"[-117.2352626288379, 32.79485122236176]",20160726_173906_0e0e
31997,"[121, 121, 119, 117, 119, 112, 118, 117, 126, ...",0,"[-121.4835244137727, 38.53386512064711]",20151030_162249_0c03
31998,"[172, 190, 202, 202, 196, 193, 188, 191, 199, ...",0,"[-118.32580026781089, 33.83093529544055]",20160715_174337_0e0e


In [8]:
data.head()

,data,labels,locations,scene_ids
0,"[206, 195, 187, 183, 177, 175, 174, 193, 198, ...",1,"[-118.40497658522878, 33.940618514147936]",20170620_175442_0e30
1,"[215, 209, 200, 196, 192, 197, 205, 168, 155, ...",1,"[-122.392469714, 37.6176425378]",20161212_180859_0e30
2,"[204, 214, 220, 219, 213, 205, 198, 193, 199, ...",1,"[-122.397578597, 37.6209247852]",20170524_181349_0e2f
3,"[179, 174, 179, 178, 173, 170, 168, 168, 168, ...",1,"[-122.214849831, 37.7203378331]",20161110_180707_0e1f
4,"[222, 222, 218, 214, 208, 205, 207, 206, 206, ...",1,"[-117.862173435, 33.6796854072]",20160813_184932_0c64


In [9]:
data.isnull().sum()

data         0
labels       0
locations    0
scene_ids    0
dtype: int64